# Clase 04 - Mapas y redes

Profesor: **Fernando Becerra**, f.becerra@udd.cl, [www.fernandobecerra.com](www.fernandobecerra.com)

En esta clase expanderemos aún más nuestro repertorio de visualizaciones aprendiendo a trabajas con mapas y redes. Para los datos geográficos trabajaremos con el paquete `geopandas`, el cual tiene algunos requerimientos extras que deben ser instalados, y los datos de redes usaremos `networkx`.

## Mapas

Importamos los paquetes estándar para comenzar a trabajar en visualización de datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

Para mapas y datos geográficos usaremos [geopandas](https://geopandas.org/en/stable/index.html), paquete que extiende los tipos de datos que puede cargar `pandas` para poder ejecutar operaciones espaciales en datos de tipo geométricos.

In [ ]:
import geopandas as gpd

Para cargar datos geográficos ocupamos la función `read_file` de geopandas. En cuanto a formato de los datos, usualmente los podemos encontrar ya sea en shapefile (`.shp`) o geojson (`.geojson` o simplemente `.json`). Para este ejemplo usaremos datos de la Región Metropolitana.

In [ ]:
rm = gpd.read_file('../../datos/RM/LIMITE_URBANO_CENSAL_C17.shp')
rm.head()

Como `geopandas` es básicamente una extensión de `pandas`, los dataframe cargados tiene funciones y métodos similares. Usemos, por ejemplo, el `.plot`.

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,8))

rm.plot(ax=ax)

plt.show()

### Graficando datos espaciales

Ahora probemos graficando datos espaciales. [Tercera dosis](https://terceradosis.cl/2022/09/16/plebiscito-de-salida-y-la-distribucion-urbana-de-la-elite-progresista-los-veinte-barrios-donde-se-concentro-el-apruebo-en-santiago/) publicó un interesante artículo/análisis del resultado del plebiscito. Lo interesante es que incluye dos visualizaciones y además comparten el código que usaron para hacer los mapas en forma de [notebook](https://colab.research.google.com/drive/1VxBXA_Dqdme4C6GLTYlywUHLOHkNyI02?usp=sharing). Así que veremos si logramos reproducir las figuras y discutiremos cómo les podemos hacer pequeñas mejoras.

Ahora tenemos que bajar los datos que los autores compartieron desde [acá](https://storage.googleapis.com/notas-blog-public/varios/sf_santiago_plebiscito.zip) y ver qué es lo que hay adentro.

In [ ]:
barrios = gpd.read_file('../../datos/sf_santiago_plebiscito/voronoi_attributes.shp')
barrios.sample(3)

### Figura 1

Como ya sabemos graficar con `geopandas`, copiemos y peguemos el código del notebook.

In [ ]:
ax = barrios.plot(column = 'apruebo', edgecolor = "black", legend = True, alpha = 0.7, scheme='fisher_jenks')

plt.show()

Hay algunas pequeñas diferencias con la [imágen publicada en el artículo](https://i0.wp.com/terceradosis.cl/wp-content/uploads/2022/09/image-12.png). Tratemos de hacerlo lo más cercano posible.

In [ ]:
ax = barrios.plot(column = 'apruebo', edgecolor = "black", legend = False, alpha = 1.0, linewidth=0.2)

plt.show()

Esta geometría se llama [celdas de Voronoi](https://en.wikipedia.org/wiki/Voronoi_diagram) y son útiles para [dividir el espacio en base a una serie de puntos](https://observablehq.com/@d3/voronoi-labels), donde el resultado es que cada celda es la más cercana sólo a uno de aquellos puntos.

Del artículo podemos sacar el tema en el que se están tratando de enfocar.

> En la Figura 1, los colores oscuros reflejan una mayor votación por la opción “rechazo”, y colores claros representan más votos para el “apruebo”

Ahora cabe la pregunta, ¿es esa la mejor forma de visualizar esos datos? ¿Qué formas alternativas se les ocurren?

Partamos por examinar los datos un poco más.

In [ ]:
barrios['apruebo'].describe()

Como aquí se quiere hacer una distinción entre apruebo/rechazo, creo que lo más conveniente sería tener distintos colores para cada opción. Para esto, yo usaría un mapa de color divergente con el límite en 0.5, que es lo que indica si gano el apruebo o el rechazo en cada lugar.

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,8))

barrios.plot(column='apruebo', edgecolor="darkgray", legend=True, alpha=1.0, ax=ax, linewidth=0.3,
             cmap='BrBG', vmin=0.3, vmax=0.7)

ax.set_axis_off()
plt.tight_layout()
plt.show()

Ahora los comparamos

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(16,8))

barrios.plot(column = 'apruebo', edgecolor = "black", legend = True, alpha = 0.7, scheme='fisher_jenks', ax=ax[0])
barrios.plot(column='apruebo', edgecolor="darkgray", legend=True, alpha=1.0, ax=ax[1], linewidth=0.3,
             cmap='BrBG', vmin=0.3, vmax=0.7)

ax[1].set_axis_off()
plt.tight_layout()
plt.show()

### Figura 2

Copiemos y peguemos el código que usaron ellos.

In [ ]:
# función para generar paleta de colores a partir de dos variables
def colorFromBivariateData(Z1,Z2,cmap1 = plt.cm.YlOrRd, cmap2 = plt.cm.PuBuGn):
    Z1_plot = np.array(255*(Z1-Z1.min())/(Z1.max()-Z1.min()), dtype = int)
    Z2_plot = np.array(255*(Z2-Z2.min())/(Z2.max()-Z2.min()), dtype = int)
    Z1_color = cmap1(Z1_plot)
    Z2_color = cmap2(Z2_plot)
    Z_color = Z1_color * Z2_color    
    return Z_color

In [ ]:
gdf_filter = barrios
pal = plt.cm.plasma
Z_color = colorFromBivariateData(gdf_filter['apruebo'], gdf_filter['nse_i'], cmap1 = pal, cmap2 = pal)
ax = gdf_filter.plot(color = Z_color, edgecolor = "black", legend = True, alpha = 0.7, scheme='fisher_jenks')

plt.show()

In [ ]:
barrios['nunoismo'] = barrios['apruebo'] * barrios['nse_i']
barrios['nunoismo'].describe()

Graficamos

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,8))

barrios.plot(column='nunoismo', edgecolor="darkgray", legend=True, alpha=1.0, ax=ax, linewidth=0.3,
             cmap='viridis')

ax.set_axis_off()
plt.tight_layout()
plt.show()

Y comparamos

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(16,8))

gdf_filter.plot(color = Z_color, edgecolor = "black", legend = True, alpha = 0.7, scheme='fisher_jenks', ax=ax[0])
barrios.plot(column='nunoismo', edgecolor="darkgray", legend=True, alpha=1.0, ax=ax[1], linewidth=0.3,
             cmap='viridis')

ax[1].set_axis_off()
plt.tight_layout()
plt.show()

¿Qué diferencias hay entre ambas representaciones? ¿Qué tan efectivas son al comunicar los datos? ¿Cuál prefieren?

### Calculando datos espaciales

Ahora carguemos más datos. ¿Se acuerdan de los datos de la Encuesta Origen Destino que usamos la clase pasada?  Bueno, carguemos la geografía para esos datos.

In [ ]:
zonas_eod = gpd.read_file('../../datos/EOD2012/Zonificacion_EOD2012.shp')
zonas_eod.head()

Y grafiquémosla

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,8))

zonas_eod.plot(ax=ax)

plt.show()

Carguemos los mismos datos de la semana pasada, que tienen la información de los viajes de las personas encuestadas

In [ ]:
viajes_persona = pd.read_csv('../../datos/eod_processed.csv')
viajes_persona = viajes_persona[pd.notnull(viajes_persona['PesoLaboral'])]
viajes_persona.head()

Filtremos sólo las zonas que están presentes en los datos de viajes y probemos intersectándolas con los datos de la Región Metropolitana

In [ ]:
zonas_con_viajes = zonas_eod[zonas_eod.ID.isin(viajes_persona.ZonaOrigen.unique())
                           & zonas_eod.ID.isin(viajes_persona.ZonaDestino.unique())]

zonas_urbanas = gpd.overlay(zonas_con_viajes, zonas_eod, how='intersection')
zonas_urbanas.plot()

Una última transformación para poder trabajar con los datos

In [ ]:
zonas_urbanas.head()

In [ ]:
zonas_urbanas = zonas_urbanas[~zonas_urbanas.Comuna_1.isin(['Pirque', 'Calera de Tango', 'Lampa', 'Colina'])]
zonas_urbanas = zonas_urbanas.set_index('ID_1')
zonas_urbanas.head()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,8))

zonas_urbanas.plot(ax=ax)

plt.show()

### Choropleth

Para hacer un chroropleth vamos a asignarle a cada zona un valor. En nuestro ejemplo, este valor será la distancia que recorren la gente de esa comuna en su viaje hacia el trabajo. Para trabajar con los datos de viajes, primero debemos transformarlos a un GeoDataFrame.

In [ ]:
origenes_viajes = gpd.GeoDataFrame(viajes_persona,
                                geometry=gpd.points_from_xy(viajes_persona['OrigenCoordX'], viajes_persona['OrigenCoordY']),
                                crs=zonas_eod.crs)
origenes_viajes.head()


Para calcular la distancia de la comuna de origen, tomaremos un promedio ponderado del atributo `DistEuclidiana` (que ya viene en los datos) de cada viaje.

In [ ]:
def weighted_mean(df, value='DistEuclidiana', weight='PesoLaboral'):
    weighted_sum = (df[value] * df[weight]).sum()
    return weighted_sum / df[weight].sum()

viajes_trabajo = viajes_persona[(viajes_persona.PropositoAgregado == 'Trabajo') &
                                (pd.notnull(viajes_persona.PesoLaboral))]
distancia_zonas = viajes_trabajo.groupby(['ZonaOrigen']).apply(weighted_mean)
distancia_zonas.name = 'distancia_al_trabajo'

In [ ]:
viajes_trabajo.head()

In [ ]:
distancia_zonas

Con los datos ya calculados ahora podemos graficar el mapa con los colores correspondientes a la distancia al trabajo.

In [ ]:
zonas_urbanas_con_distancia = zonas_urbanas.join(distancia_zonas, how='inner')

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,8))

zonas_urbanas_con_distancia.plot(
    ax=ax, 
    column='distancia_al_trabajo', 
    k=5,
    scheme='Fisher_Jenks',
    cmap='cividis_r',
    edgecolor='#abacab',
    linewidth=0.2,
    legend=True
)

plt.title('Distancia al Trabajo (Fisher Jenks)')

leg = ax.get_legend()
leg.set_bbox_to_anchor((1., 0.45, 0.2, 0.2))
ax.set_axis_off()
plt.show()

### Puntos

Ahora probaremos una variante del mapa anterior usando puntos para cada viaje. Para eso, usaremos su lugar de origen, de la misma forma que en el anterior. Para eso, primero debemos filtrar sólo los viajes que se realizan en el área urbana de la RM y sólo aquellos que son de trabajo.

In [ ]:
origenes_urbanos = gpd.sjoin(origenes_viajes.to_crs(zonas_urbanas.crs), 
                            zonas_urbanas, 
                            predicate='within', lsuffix='_l', rsuffix='_r')
origenes_a_graficar = origenes_urbanos[origenes_urbanos.PropositoAgregado == 'Trabajo']

Y lo graficamos, igual que el anterior

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(14,10))

origenes_a_graficar.plot(column='DistEuclidiana', 
       ax=ax, 
       marker='.', 
       markersize=20,  
       cmap='cividis_r', 
       legend=True)

ax.set_axis_off()

Podemos agregarle el mapa de la RM de fondo para darle más contexto.

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(14,10))

zonas_urbanas.plot(ax=ax, color='#efefef', edgecolor='#abacab', linewidth=1, alpha=0.5)

origenes_a_graficar.plot(column='DistEuclidiana', 
       ax=ax, 
       marker='.', 
       markersize=20,  
       cmap='cividis_r', 
       legend=True)

ax.set_axis_off()

Y podemos comparar ambos mapas.

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(18,12))

zonas_urbanas.plot(ax=ax[0], color='#efefef', edgecolor='#abacab', linewidth=1, alpha=0.5)
origenes_urbanos[origenes_urbanos.PropositoAgregado == 'Trabajo'].plot(
    column='DistEuclidiana', 
    ax=ax[0], 
    marker='.', 
    markersize=20,  
    cmap='cividis_r')

zonas_urbanas.join(distancia_zonas, how='inner').plot(ax=ax[1],
                                          column='distancia_al_trabajo', 
                                          k=5,
                                          scheme='Fisher_Jenks',
                                          cmap='cividis_r',
                                          edgecolor='#abacab',
                                          linewidth=0.2)

ax[0].set_axis_off()
ax[1].set_axis_off()

### Heatmap

Para graficar un heatmap sobre un mapa, ocuparemos una función especial que calcule el mapa de calor primero, y después lo sobrepondremos al mapa geográfico.

In [ ]:
from KDEpy import FFTKDE

def build_heatmap_data(trips):
    # hacemos una lista con las coordenadas de los viajes
    point_coords = np.vstack([trips.geometry.x, trips.geometry.y]).T
    # instanciamos la Fast-Fourier Transform Kernel Density Estimation
    kde = FFTKDE(bw=1e-2, norm=2, kernel='cosine')
    # La variable grid_points define la cantidad de puntos en el espacio en el que se estimará la densidad
    grid_points = 2**6  
    # ajustamos la distribución.
    # ¡Noten que el parámetro weights recibe el factor de expansión!
    grid, points = kde.fit(point_coords, weights=trips['PesoLaboral'].values).evaluate(grid_points)
    x, y = np.unique(grid[:, 0]), np.unique(grid[:, 1])
    z = points.reshape(grid_points, grid_points).T
    return x, y, z

Creamos el mapa de calor y probamos como se ve.

In [ ]:
test_heatmap = build_heatmap_data(origenes_urbanos)
sns.heatmap(test_heatmap[2], cmap='magma_r')

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(14,10))

zonas_urbanas.plot(ax=ax, color='#efefef', edgecolor='#abacab', linewidth=0.5)

n_levels = 20
ax.contourf(test_heatmap[0], test_heatmap[1], test_heatmap[2], n_levels, alpha=0.8, cmap="magma_r")

ax.set_axis_off()


In [ ]:
fig, ax = plt.subplots(1,1, figsize=(14,10))

zonas_urbanas.plot(ax=ax, color='#efefef', edgecolor='#abacab', linewidth=0.5)

n_levels = 20
masked_z = np.ma.array(test_heatmap[2], mask=test_heatmap[2] < 1.0)
ax.contourf(test_heatmap[0], test_heatmap[1], masked_z, n_levels, alpha=0.8, cmap="magma_r")

ax.set_axis_off()

### Redes

Primero, importamos [networkx](https://networkx.org/documentation/stable/index.html), que es el paquete que nos facilitará la vida al momento de graficar redes.

In [ ]:
import networkx as nx

Cargamos los datos, que son las interacciones de los personajes de la novela Les Miserables.

In [ ]:
ls = pd.read_csv('../../datos/lesmiserables.csv')
ls.head()

Creamos la read a partir de esos datos.

In [ ]:
network = nx.from_pandas_edgelist(ls)

Y la dibujamos

In [ ]:
nx.draw(network)

In [ ]:
nx.draw_networkx(network)

In [ ]:
nx.draw_kamada_kawai(network)

También se puede graficar de forma circular. Para eso primero calculamos las posiciones de los nodes y edges a parte.

In [ ]:
positions = nx.circular_layout(network)

fig, ax = plt.subplots(1,1, figsize=(10,10))

nx.draw_networkx_nodes(network, pos=positions, node_size=30, ax=ax)
nx.draw_networkx_edges(network, pos=positions, alpha=0.1, ax=ax)

ax.set_axis_off()

De vuelta a la EOD. Filtremos los viajes que se realizan en el área urbana y creemos una matriz similar a la anterior de Los Miserables, pero usando zonas de origen y destino de los viajes.

In [ ]:
viajes_urbanos = viajes_trabajo[viajes_trabajo.ZonaOrigen.isin(zonas_urbanas.index) & 
                                viajes_trabajo.ZonaDestino.isin(zonas_urbanas.index)].copy()


matrices = (
    viajes_urbanos[pd.notnull(viajes_urbanos.PesoLaboral)]
        .groupby(['ZonaOrigen', 'ZonaDestino'])
        .agg(n_viajes=('PesoLaboral', 'sum'))
)

matrices

Las posiciones de las zonas ya están calculadas, asi que ocupamos las coordenadas de los `centroid` de los polígonos.

In [ ]:
centroids = zonas_urbanas.centroid
centroids.head()

In [ ]:
node_positions = dict(zip(centroids.index, zip(centroids.x, centroids.y)))
node_positions

Creamos un gráfico de redes a partir de esos datos, de la misma forma que lo hicimos con los otros datos.

In [ ]:
graph_matrix = matrices[(matrices.n_viajes >= matrices.n_viajes.quantile(0.95))].reset_index()
graph = nx.from_pandas_edgelist(graph_matrix, 
                                source='ZonaOrigen', target='ZonaDestino', 
                                edge_attr='n_viajes',
                                create_using=nx.DiGraph())

Y finalmente lo graficamos sobre el mapa.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16,16))

zonas_urbanas.plot(ax=ax, facecolor='#efefef', edgecolor='#666666', alpha=0.1)
nx.draw(graph, ax=ax, pos=node_positions, node_size=30, edge_color='darkgray')

ax.set_axis_off()

Además podemos usar distintos anchos para los edges, dependiendo del número de viajes realizados de una zona a otra.

In [ ]:
edge_width = graph_matrix['n_viajes'] / graph_matrix['n_viajes'].max() * 7

fig, ax = plt.subplots(1, 1, figsize=(16,16))

zonas_urbanas.plot(ax=ax, facecolor='#efefef', edgecolor='#666666', alpha=0.1)
nx.draw_networkx(graph, ax=ax, pos=node_positions, node_size=30, edge_color='darkgray',
                 with_labels=False, width=edge_width)

ax.set_axis_off()